In this example, we implement a fraud detection system using PySpark and Random Forest Classifier. 

#### Installation of Pyspark package:

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 19.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=4a31f8cf6fa3acc594bf61679549065d82974b6688e3445aa6db61b8448b9a91
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


#### Installation of relevant packages:

In [2]:
import pandas as pd
import warnings
warnings.simplefilter(action ='ignore', category = FutureWarning)
pd.set_option('display.max_columns', None) # To see all columns
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext.getOrCreate() 
spark = SparkSession.builder.appName("Fraud Detection").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/06 19:37:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Load the data in pandas dataframe form:

In [3]:
df_raw = spark.read.format("csv").option("header","true")\
.load("../input/creditcardfraud/creditcard.csv")
df_raw.sample(False, 0.00005, seed= 222).toPandas()

23/04/06 19:37:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,20810,-1.32407076104731,0.39226524482227,2.45388276243653,-0.777917572527888,0.477728391919903,-0.065639521424042,0.784531476137707,-0.0294323524539612,1.3304020578933,-1.18841515286498,2.02820943964263,-2.03790088031151,0.33895625479595,1.26563520731193,-2.35062284479114,0.133783128474008,0.166870227376042,-0.411414638574038,-1.59112415792566,-0.347641344536826,-0.37967280356364,-0.666759979649487,0.0418841910198923,0.140117069082837,0.327299392332492,-0.128318625368819,-0.303035302832668,-0.223775402578992,29.99,0
1,22464,-1.29615140749684,1.8207792839607,0.923497081103908,0.867634580742225,-0.0528253931619091,-0.853530125085793,0.614694762167824,-0.129949727464723,1.64774969472327,0.876394473073704,0.391618205912992,-3.02899538586015,0.835522542687217,1.62247203693637,0.445591289939061,-0.395598296772,0.54945038355868,0.405348870823401,0.0906932839065072,0.474216907925097,-0.198961132845738,0.147995454660481,-0.120592938613445,0.306939570358327,-0.0393474555513816,-0.33181330386076,0.835012402242817,0.476200919191581,18.43,0
2,38366,0.909185098730668,-0.683311939904696,1.06291560342406,0.896904056400123,-1.30447101514779,-0.285876153851401,-0.485578198501518,0.0666763749862112,1.02013700297509,-0.303319853197091,-0.821865128630231,0.12462207018221,-0.52941034420468,-0.261573408099837,0.292797667466465,0.216233151013316,-0.122510463069827,-0.157480678288715,-0.0487441130511958,0.157917154120779,-0.0161178277808101,-0.208202428214607,-0.0824256770328227,0.430865632091007,0.158623932562846,0.40881372354036,-0.0221629311139634,0.0493302022650248,152.09,0
3,53314,1.09609566451139,-0.081471594201628,0.444016490361446,1.36144276247599,-0.480774467313936,-0.0535777843717323,-0.238308179912908,0.295995697269603,0.496776885788334,0.134944222481358,0.804694850986288,-0.398922964868424,-3.42036559212194,0.906637683609856,-0.461862075165469,-0.437491336217184,0.268747074242382,-0.160323399334514,-0.135411958753111,-0.380039925711771,-0.070239071682282,-0.167256979546703,-0.0158423308955405,0.145674627738277,0.496599227678627,-0.332164359763401,0.0148327471154731,0.00179649409057866,12.99,0
4,58505,-0.36074504838322,0.27394485802079,1.40271774913656,-0.876609131449862,-0.494343442307687,-0.413083092309656,0.362972759597238,0.0295384408094224,-1.94963886856537,0.573692399130834,0.655883928892333,-0.874049883352861,-1.5915034463653,0.790838158027074,0.634483569893616,-1.49831272273025,0.20621490985088,1.47631150335466,1.13348689447765,-0.294290301517989,-0.722012908233125,-1.85111321773583,0.0546133167368721,-0.0828801423702857,-0.261091196888113,0.813585890755777,-0.0338673254355134,0.0496844496525934,34.85,0
5,73201,-0.407525487594329,0.947976203307134,1.12448620177617,-0.169994129655692,0.332677619678395,-0.0167052747594617,0.471640790524572,0.287816799843317,-0.489023460800919,-0.360754188332176,1.24756329958886,0.0633215684245002,-0.910015821061456,-0.0286095487834816,0.577036151348421,0.428981000912058,-0.027475561961835,0.136427087869869,-0.0304362402169674,0.0469223673053849,-0.215717601978188,-0.603145517619701,-0.0197216538609954,-0.386728063054296,-0.251780448355723,0.107663272966509,0.250480890958882,0.0794245239578177,6.45,0
6,77065,-1.15312236813067,0.778329173832611,0.907361007496881,-1.89897319774874,-0.363549878159229,-1.43796415746604,0.635062594273106,0.148761975525196,0.401993217929443,-1.78377858674973,-0.508180638408115,0.508851199414633,0.0232931947903972,0.535829893156595,0.98466324824825,-0.280940203490784,-0.311562526437929,0.0958199107667548,-0.0123364420713897,-0.219814092332548,0.129863263134295,0.209521841255965,-0.202926414845482,0.397593366451805,0.349009339177686,-0.850265615387006,-0.137198431388385,0.0317604758096658,39.75,0
7,83126,-1.63427828393981,1.61621324901074,0.473151775338067,1.25063546632682,-0.260168387100038,-0.512252491114008,0.851363029452164,0.315512468392413,-0.809341

#### Cast "Class" column to integer and the rest to double:

In [4]:
from pyspark.sql.functions import col
cols= [col(c).cast("double") for c in df_raw.columns[:-1]]
df=df_raw.select(df_raw.Class.cast("int"), * cols)
df.printSchema()

root
 |-- Class: integer (nullable = true)
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nu

#### Remove the rows with missing values:

In [5]:
df= df.dropna()

#### Separate the features from the target variable:

In [6]:
from pyspark.ml.linalg import DenseVector
rdd_ml = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))
df_ml = spark.createDataFrame(rdd_ml, ["labels", "features"])
df_ml.sample(False, 0.00005, seed=222).toPandas()

,labels,features
0,0,"[20810.0, -1.32407076104731, 0.39226524482227,..."
1,0,"[22464.0, -1.29615140749684, 1.8207792839607, ..."
2,0,"[38366.0, 0.909185098730668, -0.68331193990469..."
3,0,"[53314.0, 1.09609566451139, -0.081471594201628..."
4,0,"[58505.0, -0.36074504838322, 0.27394485802079,..."
5,0,"[73201.0, -0.407525487594329, 0.94797620330713..."
6,0,"[77065.0, -1.15312236813067, 0.778329173832611..."
7,0,"[83126.0, -1.63427828393981, 1.61621324901074,..."
8,0,"[119353.0, -1.2143433353886, -0.02217503918258..."
9,0,"[121831.0, 2.08756582798647, -1.13432993512729..."


#### Split the data and train the model with Random Forest Classifier:

In [7]:
train, test= df_ml.randomSplit([0.8,0.2], seed=222)
from pyspark.ml.classification import RandomForestClassifier
clf= RandomForestClassifier(labelCol="labels", featuresCol="features")
model= clf.fit(train)

#### Get the predictions from the model:

In [8]:
predictions = model.transform(test)
predictions.show()

+------+--------------------+--------------------+--------------------+----------+
|labels|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|[7.0,-0.894286082...|[19.9951490806304...|[0.99975745403152...|       0.0|
|     0|[18.0,0.247491127...|[19.9951490806304...|[0.99975745403152...|       0.0|
|     0|[23.0,-0.41428881...|[19.9951490806304...|[0.99975745403152...|       0.0|
|     0|[23.0,1.173284610...|[19.9951490806304...|[0.99975745403152...|       0.0|
|     0|[26.0,-0.53538776...|[19.9951490806304...|[0.99975745403152...|       0.0|
|     0|[34.0,0.201685891...|[19.9951490806304...|[0.99975745403152...|       0.0|
|     0|[37.0,1.295667620...|[19.9951490806304...|[0.99975745403152...|       0.0|
|     0|[38.0,0.158331782...|[19.9951490806304...|[0.99975745403152...|       0.0|
|     0|[39.0,-0.55406456...|[19.9953970732936...|[0.99976985366468...|       0.0|
|   

#### Calculate the accuracy of the model:

In [9]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator= MulticlassClassificationEvaluator(metricName="accuracy", labelCol="labels", predictionCol="prediction")
accuracy= evaluator.evaluate(predictions)
print("Accuracy: ", accuracy)

Accuracy:  0.9993842147858827
